# Load Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [ ]:
df = pd.read_csv('loan.csv')

# Data Preprocess

## drop nan
* In this problem, dropping nan just makes 614 to 480, that is OK
* But in lab5, dropping nan makes 10000 to 17000, that is unacceptable

In [ ]:
df.drop("Loan_ID", axis=1, inplace=True) # this column is useless
print(df.isnull().sum()) # Check how many nans there are in df

In [ ]:
# Task1 deal with NULL rows, you can either choose to drop them or replace them with mean or other value
df = df.dropna(axis=0, how='any', inplace=False) # drop NaN

## Encode categorical features
* Yaning Yang told me we could use dummy variables, but I didn't learn it well

In [ ]:
# Task2 deal with categorical features
df.Gender=df.Gender.map({'Male':1,'Female':0})
df.Married=df.Married.map({'Yes':1,'No':0})
df.Dependents=df.Dependents.map({'3+':3,'2':2,'1':1,'0':0})
df.Education=df.Education.map({'Graduate':1,'Not Graduate':0})
df.Self_Employed=df.Self_Employed.map({'Yes':1,'No':0})
df.Property_Area=df.Property_Area.map({'Urban':2,'Semiurban':1,'Rural':0})
df.Loan_Status=df.Loan_Status.map({'Y':1,'N':-1})

## descriptive statistics

In [ ]:
print(df.info())

In [ ]:
print(df["Loan_Status"].value_counts())

In [ ]:
df.describe()

## histogram

In [ ]:
df = df.to_numpy()

In [ ]:
plt.hist(df[:,5], density = True, bins = 10, facecolor="blue", edgecolor="black")
plt.xlabel('applicant income') 
plt.ylabel('frequency') 

In [ ]:
plt.hist(df[:,6], density = True, bins = 10, facecolor="blue", edgecolor="black")
plt.xlabel('coapplicant income') 
plt.ylabel('frequency') 

In [ ]:
plt.hist(df[:,7], density = True, bins = 10, facecolor="blue", edgecolor="black")
plt.xlabel('loan amount') 
plt.ylabel('frequency') 

In [ ]:
plt.hist(df[:,8], density = True, bins = 10, facecolor="blue", edgecolor="black")
plt.xlabel('loan amount term') 
plt.ylabel('frequency') 

# split the train set and the test set
* normalization vs standardization
  * The former one usually refers to min-max-normalization
  * The latter one usually refers to Z-score-standardization
  * distinction
    * Normalization is used when the data doesn't have Gaussian distribution whereas Standardization is used on data having Gaussian distribution.
    * Normalization scales in a range of [0,1] or [-1,1]. Standardization is not bounded by range.
    * Normalization is highly affected by outliers. Standardization is slightly affected by outliers.
    * Normalization is considered when the algorithms do not make assumptions about the data distribution. Standardization is used when algorithms make assumptions about the data distribution.
* 先划分训练集测试集还是先归一化或标准化？
  * 严格来说，应该先划分，否则测试集就用到了训练集的信息
  * 在lab1和lab5中，我都是先划分再min-max归一化
> ***Keep it simple, when you get too complex you forget the obvious***

In [ ]:
# 方法1：用min-max来标准化
def min_max_normalization(data):
    return (data - np.min(data, axis = 0))/(np.max(data, axis = 0) - np.min(data, axis = 0))
    
# 方法2：用Z-score来标准化
def Z_score_standardization(data):
    mu = np.mean(data, axis = 0)
    sigma = np.std(data, axis = 0)
    return (data - mu) / sigma

In [ ]:
# Task3 split the dataset into X_train, X_test, y_train, y_test

# Shuffle the rows of the array
np.random.seed(0)  # Set a seed for reproducibility
df = df[np.random.permutation(df.shape[0])]

# Split the array into two
split = int(0.8 * df.shape[0])  # Calculate the index at which to split
df1 = df[:split]  # Select rows 0 to split-1
df2 = df[split:]  # Select rows split to end

X_train = df1[:,0:11]
Y_train = df1[:,11]
X_train = min_max_normalization(X_train)
X_train_design = np.insert(X_train, 0, 1, axis = 1)

X_test = df2[:,0:11]
Y_test = df2[:,11]
X_test = min_max_normalization(X_test)
X_test_design = np.insert(X_test, 0, 1, axis = 1)

# Train

In [ ]:
# super parameter
eta = 0.1					    # 学习率
epochs = 10 ** 4				# 迭代上限
epsilon = 10 ** -5				# 梯度模长的上限

In [ ]:
# Task4 train your model and plot the loss curve of training

# 初始点
w_k = np.zeros(X_train_design.shape[1])

# coherent to formula
X = X_train_design
y = Y_train

def sigmoid(x):
    return 1/(1+np.exp(-x))

def h(x):
	return sigmoid(np.dot(x,w_k))

def rhd(w):
	return np.mean(np.log(1+np.exp(-y*(X@w))))

def d_rhd(w):
	return np.array([np.mean(-y*X[:, i]*np.exp(-y*(X@w))/(1+np.exp(-y*(X@w)))) for i in range(12)])

loss = np.zeros(math.floor(epochs))
for i in range(math.floor(epochs)): # 时间复杂度 O(n)
	loss[i] = rhd(w_k)
	d_rhd_wk = d_rhd(w_k) # d_rhd_wk is the gradient of rhd
	if np.linalg.norm(d_rhd_wk) < epsilon:
		break
	w_k = w_k - eta * d_rhd_wk

print("梯度下降法终止时的梯度的范数为：{}".format(np.linalg.norm(d_rhd_wk)))
plt.plot(loss[0:i+1])
plt.xlabel('epochs')
plt.ylabel('loss')

In [ ]:
print("w: {}".format(w_k))

In [ ]:
# accuracy of the train set
y_prediction = w_k @ X_train_design.T
y_prediction = np.where(y_prediction > 0, +1, -1)
print("accuracy of the train set: {}".format(1 - np.sum(np.abs(y_prediction - Y_train) / 2) / Y_train.shape[0]))

# Test

In [ ]:
# accuracy of the test set
z = w_k @ X_test_design.T
y_prediction = np.where(z > 0, +1, -1)
print("accuracy of the test set: {}".format(1 - np.sum(np.abs(y_prediction - Y_test) / 2) / Y_test.shape[0]))

# Other model

## least square/pseudo inverse

In [ ]:
# calculate w
pinvX = np.linalg.pinv(X_train_design)   # calculate pseudo inverse
w = pinvX @ Y_train              
print("w: {}".format(w))

# accuracy of train set
z = w @ X_train_design.T
y_prediction = np.where(z > 0, +1, -1)
print("accuracy of train set: {}".format(1 - np.sum(np.abs(y_prediction - Y_train) / 2) / Y_train.shape[0]))

# accuracy of test set
z = w @ X_test_design.T
y_prediction = np.where(z > 0, +1, -1)
print("accuracy of test set: {}".format(1 - np.sum(np.abs(y_prediction - Y_test) / 2) / Y_test.shape[0]))

## 用sklearn实现least square和Logistic regression

### least square/pseudo inverse

In [ ]:
from sklearn.linear_model import LinearRegression

# linear regression
Least_Square = LinearRegression().fit(X_train, Y_train)

# print coef and intercept
w_LS = Least_Square.coef_
b_LS = Least_Square.intercept_
print("w: {}".format(Least_Square.coef_))
print("b: {}".format(Least_Square.intercept_))

# accuracy of train set
y_prediction = Least_Square.predict(X_train)
y_prediction = np.where(y_prediction > 0, +1, -1)
print("accuracy of train set: {}".format(1 - np.sum(np.abs(y_prediction - Y_train) / 2) / Y_train.shape[0]))

# accuracy of test set
y_prediction = Least_Square.predict(X_test)
y_prediction = np.where(y_prediction > 0, +1, -1)
print("accuracy of test set: {}".format(1 - np.sum(np.abs(y_prediction - Y_test) / 2) / Y_test.shape[0]))

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# logistic regression
logistic_regression = LogisticRegression(penalty='none', random_state=0).fit(X_train, Y_train)

# print coef and intercept
w_logistic_regression = logistic_regression.coef_
b_logistic_regression = logistic_regression.intercept_
print("w: {}".format(logistic_regression.coef_))
print("b: {}".format(logistic_regression.intercept_))

# accuracy of train set
y_prediction = logistic_regression.predict(X_train)
print("accuracy of train set: {}".format(1 - np.sum(np.abs(y_prediction - Y_train) / 2) / Y_train.shape[0]))

# accuracy of test set
y_prediction = logistic_regression.predict(X_test)
print("accuracy of test set: {}".format(1 - np.sum(np.abs(y_prediction - Y_test) / 2) / Y_test.shape[0]))

# 总结
* $epoch = 10^4$时，正确率已经到达极限了
  * 正确率的极限只有80%左右
  * 要想更好，需要用特征转换等方法
* w中绝对值最大的竟然是Property_Area，这是不是地域歧视啊；绝对值最小的是feature 6
* 类别不是特别不平衡的时候不要用$N+/N-$代替1/2，否则会过拟合